<a href="https://colab.research.google.com/github/zhenhua-wang/Inference-on-Friendship-and-Personality/blob/master/CDS_vol4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Load Data and Pre-processing

In [37]:
import pandas as pd
import numpy as np

In [14]:
rating = pd.read_csv('/content/drive/MyDrive/NBA Stats/2k_ratings.csv')
feature = pd.read_csv('/content/drive/MyDrive/NBA Stats/vol4/vol4_data.csv')

In [15]:
# Clean Name
rating['Player'] = rating['Player'].str.split(' https', expand=True)[0]

In [21]:
# Construct design matrix
df = feature.merge(rating, how='inner', left_on='Name', right_on='Player')

In [22]:
df

,Unnamed: 0,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,Name,Data_type,GS_mean,MIN_mean,FGM_mean,FGA_mean,FG3M_mean,FG3A_mean,FTM_mean,FTA_mean,OREB_mean,DREB_mean,AST_mean,STL_mean,BLK_mean,TOV_mean,PF_mean,PTS_mean,TEAM,W,L,WIN_PCT,PS,PA,Conference,Rank,Player,Team,Pos.,OVR,Height,Build
0,0,203500,2020-21,0,1610612740,NOP,27.0,58,58,1605.0,189,308,0.614,0,3,0.000,60,135,0.444,213,301,514,111,54,38,78,113,438,Steven Adams,SeasonTotalsRegularSeason,1.000000,27.672414,3.258621,5.310345,0.000000,0.051724,1.034483,2.327586,3.672414,5.189655,1.913793,0.931034,0.655172,1.344828,1.948276,7.551724,New Orleans Pelicans,31,41,0.431,114.6,114.9,Western,100,Steven Adams,Pelicans,C,80,"6'11""",Paint Beast
1,1,1628389,2020-21,0,1610612748,MIA,23.0,64,64,2143.0,456,800,0.570,2,8,0.250,283,354,0.799,142,431,573,346,75,66,169,145,1197,Bam Adebayo,SeasonTotalsRegularSeason,1.000000,33.484375,7.125000,12.500000,0.031250,0.125000,4.421875,5.531250,2.218750,6.734375,5.406250,1.171875,1.031250,2.640625,2.265625,18.703125,Miami Heat*,40,32,0.556,108.1,108.0,Eastern,32,Bam Adebayo,Heat,C,87,"6'9""",Interior Force
2,2,1629638,2020-21,0,1610612740,NOP,22.0,46,13,1007.0,192,458,0.419,76,219,0.347,48,66,0.727,13,131,144,102,47,22,69,88,508,Nickeil Alexander-Walker,SeasonTotalsRegularSeason,0.282609,21.891304,4.173913,9.956522,1.652174,4.760870,1.043478,1.434783,0.282609,2.847826,2.217391,1.021739,0.478261,1.500000,1.913043,11.043478,New Orleans Pelicans,31,41,0.431,114.6,114.9,Western,222,Nickeil Alexander-Walker,Pelicans,PG,76,"6'5""",Playmaker
3,3,1628960,2020-21,0,1610612763,MEM,25.0,50,38,1260.0,173,414,0.418,107,274,0.391,79,91,0.868,19,141,160,108,46,8,48,71,532,Grayson Allen,SeasonTotalsRegularSeason,0.760000,25.200000,3.460000,8.280000,2.140000,5.480000,1.580000,1.820000,0.380000,2.820000,2.160000,0.920000,0.160000,0.960000,1.420000,10.640000,Memphis Grizzlies*,38,34,0.528,113.3,112.3,Western,179,Grayson Allen,Grizzlies,SG,77,"6'4""",3PT Specialist
4,4,203937,2020-21,0,1610612763,MEM,27.0,69,69,1887.0,308,658,0.468,94,261,0.360,144,184,0.783,52,344,396,250,84,57,86,120,854,Kyle Anderson,SeasonTotalsRegularSeason,1.000000,27.347826,4.463768,9.536232,1.362319,3.782609,2.086957,2.666667,0.753623,4.985507,3.623188,1.217391,0.826087,1.246377,1.739130,12.376812,Memphis Grizzlies*,38,34,0.528,113.3,112.3,Western,86,Kyle Anderson,Grizzlies,SF,81,"6'9""",Mid-Range Shot Creator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,355,1626174,2020-21,0,1610612745,HOU,25.0,41,41,1326.0,329,640,0.514,77,206,0.374,125,198,0.631,76,319,395,71,34,48,80,88,860,Christian Wood,SeasonTotalsRegularSeason,1.000000,32.341463,8.024390,15.609756,1.878049,5.024390,3.048780,4.829268,1.853659,7.780488,1.731707,0.829268,1.170732,1.951220,2.146341,20.975610,Houston Rockets,17,55,0.236,108.8,116.7,Western,51,Christian Wood,Rockets,PF,84,"6'10""",Inside-Out Scorer
323,356,201152,2020-21,0,1610612741,CHI,32.0,68,23,1652.0,370,662,0.559,12,45,0.267,71,113,0.628,168,255,423,291,74,40,137,152,823,Thaddeus Young,SeasonTotalsRegularSeason,0.338235,24.294118,5.441176,9.735294,0.176471,0.661765,1.044118,1.661765,2.470588,3.750000,4.279412,1.088235,0.588235,2.014706,2.235294,12.102941,Chicago Bulls,31,41,0.431,110.7,111.6,Eastern,71,Thaddeus Young,Bulls,PF,82,"6'8""",Inside-Out Scorer
324,357,1629027,2020-21,0,1610612737,ATL,22.0,63,63,2125.0,487,1112,0.438,136,397,0.343,484,546,0.886,38,207,245,594,53,12,261,111,1594,Trae Young,SeasonTotalsRegularSeason,1.000000,33.730159,7.730159,17.650794,2.158730,6.301587,7.682540,8.666667,0.603175,3.285714,9.428571,0.841270,0.190476,4.142857,1.761905,25.301587,Atlanta Hawks*,41,31,0.569,113.7,111.4,Eastern,27,Trae Young,Hawks,PG,87,"6'1""",

## 2. Split Train, Test, Validation

In [29]:
feature_col = ['FG_PCT','FG3_PCT','FT_PCT','GS_mean','MIN_mean','FGM_mean','FGA_mean','FG3M_mean',
              'FG3A_mean','FTM_mean','FTA_mean','OREB_mean','DREB_mean','AST_mean','STL_mean','BLK_mean',
              'TOV_mean','PF_mean','PTS_mean','TEAM_ABBREVIATION','W','L','WIN_PCT','PS','PA','Conference']
label_col = 'OVR'

In [30]:
# Split dataset
from sklearn.model_selection import train_test_split
xtrain = df[feature_col]
labels = df[label_col]
x, x_test, y, y_test = train_test_split(xtrain,labels,test_size=0.2,train_size=0.8)
x_train, x_cv, y_train, y_cv = train_test_split(x,y,test_size = 0.25,train_size =0.75)

In [33]:
x_train

,FG_PCT,FG3_PCT,FT_PCT,GS_mean,MIN_mean,FGM_mean,FGA_mean,FG3M_mean,FG3A_mean,FTM_mean,FTA_mean,OREB_mean,DREB_mean,AST_mean,STL_mean,BLK_mean,TOV_mean,PF_mean,PTS_mean,TEAM_ABBREVIATION,W,L,WIN_PCT,PS,PA,Conference
144,0.500,0.000,0.563,0.090909,9.727273,0.818182,1.636364,0.000000,0.090909,0.818182,1.454545,1.000000,1.363636,1.181818,0.181818,0.363636,0.181818,1.363636,2.454545,POR,42,30,0.583,116.1,114.3,Western
149,0.432,0.363,0.781,0.710145,30.811594,4.579710,10.594203,2.028986,5.594203,0.724638,0.927536,0.550725,2.797101,3.492754,1.188406,0.260870,1.144928,2.391304,11.913043,ATL,41,31,0.569,113.7,111.4,Eastern
319,0.611,0.294,0.698,1.000000,33.213115,10.393443,17.000000,0.163934,0.557377,6.049180,8.672131,2.737705,4.491803,3.704918,0.934426,0.639344,2.737705,2.213115,27.000000,NOP,31,41,0.431,114.6,114.9,Western
272,0.447,0.348,0.848,0.080000,17.420000,3.060000,6.840000,0.940000,2.700000,1.680000,1.980000,0.920000,2.880000,1.300000,0.600000,0.080000,1.140000,1.900000,8.740000,PHX,51,21,0.708,115.3,109.5,Western
131,0.512,0.394,0.892,1.000000,32.483871,7.629032,14.903226,1.322581,3.354839,2.935484,3.290323,1.000000,5.758065,3.548387,0.887097,0.822581,1.725806,1.919355,19.516129,PHI,49,23,0.681,113.6,108.1,Eastern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.508,0.125,0.816,0.090909,8.212121,1.878788,3.696970,0.030303,0.242424,0.939394,1.151515,1.424242,2.454545,0.303030,0.060606,0.181818,0.363636,0.878788,4.727273,DAL,42,30,0.583,112.4,110.2,Western
59,0.458,0.273,0.769,0.024390,6.853659,0.804878,1.756098,0.219512,0.804878,0.243902,0.317073,0.268293,0.951220,0.341463,0.268293,0.048780,0.341463,0.439024,2.073171,DEN,47,25,0.653,115.1,110.1,Western
288,0.459,0.386,0.868,1.000000,35.781250,9.453125,20.593750,2.921875,7.578125,4.609375,5.312500,0.781250,6.593750,4.312500,1.171875,0.484375,2.671875,1.906250,26.437500,BOS,36,36,0.500,112.6,111.2,Eastern
199,0.375,0.248,0.641,0.056604,15.433962,1.811321,4.830189,0.584906,2.358491,0.773585,1.207547,0.584906,2.075472,1.264151,0.698113,0.226415,0.622642,1.169811,4.981132,CHA,33,39,0.458,109.5,111.4,Eastern


## 3. Modeling

In [34]:
!pip install catboost
import catboost

     |████████████████████████████████| 67.3MB 57kB/s 


In [42]:
# categorical_features_indices = np.where(x_train.dtypes != np.float)[0]
categorical_features_indices = [19, 25]

In [43]:
# Model params
params_1 = {
    'iterations': 10000,
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'verbose': 10,
    'random_seed': 24,
    # 'depth':6, 
    'learning_rate':0.01,
    'use_best_model': True,
    'l2_leaf_reg': 10,
    'bagging_temperature': 3,
    'od_type': "Iter",
    'od_wait': 100
}

cat1 = catboost.CatBoostRegressor(**params_1)
cat1.fit(x_train, y_train,
             eval_set = (x_test, y_test),
             use_best_model = True,
             cat_features = categorical_features_indices,
             plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 5.6559710	test: 7.1639240	best: 7.1639240 (0)	total: 64.3ms	remaining: 10m 43s
10:	learn: 5.4421691	test: 6.9071496	best: 6.9071496 (10)	total: 143ms	remaining: 2m 9s
20:	learn: 5.2331266	test: 6.6463937	best: 6.6463937 (20)	total: 206ms	remaining: 1m 38s
30:	learn: 5.0549053	test: 6.4490390	best: 6.4490390 (30)	total: 300ms	remaining: 1m 36s
40:	learn: 4.8714311	test: 6.2184481	best: 6.2184481 (40)	total: 348ms	remaining: 1m 24s
50:	learn: 4.7132022	test: 6.0316490	best: 6.0316490 (50)	total: 390ms	remaining: 1m 16s
60:	learn: 4.5433036	test: 5.8170082	best: 5.8170082 (60)	total: 430ms	remaining: 1m 10s
70:	learn: 4.3839575	test: 5.6226401	best: 5.6226401 (70)	total: 471ms	remaining: 1m 5s
80:	learn: 4.2437337	test: 5.4566176	best: 5.4566176 (80)	total: 550ms	remaining: 1m 7s
90:	learn: 4.1081665	test: 5.2826955	best: 5.2826955 (90)	total: 613ms	remaining: 1m 6s
100:	learn: 3.9735568	test: 5.1096857	best: 5.1096857 (100)	total: 653ms	remaining: 1m 4s
110:	learn: 3.8543258	te

## 4. Evaluation

In [46]:
# Evaluation
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, auc, roc_curve, precision_recall_curve, r2_score, mean_squared_error
import seaborn as sns

def eval_reg(y_test, y_pred):
  '''
  This function evaluates the regression model performance
  in terms of RMSE, R2
  
  Parameters:
  y_test (array):     Test label
  y_pred (array):     Model prediction

  Returns:
  Evaluation information printed
  array contains [RMSE, r2]
  '''
  RMSE = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  print('RMSE: \t', str(RMSE))
  print('r2: \t', str(r2))
  return [RMSE, r2]

In [44]:
cat1_regr = cat1.predict(x_cv)

In [47]:
eval_reg(y_cv, cat1_regr)

RMSE: 	 6.514924113109625
r2: 	 0.8144980917298721


[6.514924113109625, 0.8144980917298721]